In [12]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import os

In [17]:
os.environ['JAVA_HOME'] = "C:/Program Files/Java/jdk-11"
spark = SparkSession.builder.appName('test_file').master("local").getOrCreate()

In [3]:
src_path = r"C:\Users\chipp\OneDrive\Desktop\Training Folder\Pyspark_Training\pyspark_training\testing_framework\data\src.csv"

src_df = spark.read.format('csv') \
                .option('header', True)\
                .option('inferSchema', True)\
                .load(src_path)

In [4]:
print(src_df.columns)

['CinemaCode', 'CinemaName', 'MovieId', 'TicketCount', 'TicketPrice', 'TransactionId', 'TransactionDt']


In [19]:
src_df.show()

+----------+--------------------+-------+-----------+-----------+--------------------+-------------------+
|CinemaCode|          CinemaName|MovieId|TicketCount|TicketPrice|       TransactionId|      TransactionDt|
+----------+--------------------+-------+-----------+-----------+--------------------+-------------------+
|     BLSCM|Bleecker Street C...| 413992|          3|        3.2|9C78647F-2E38-4AB...|2018-01-01 17:23:10|
|     BTHTR|       Booth Theater| 374471|          2|        6.4|A7C0D951-74DA-49E...|2018-01-01 14:51:48|
|     BTHTR|       Booth Theater| 439502|          3|       10.4|6E5B857C-CB73-4E1...|2018-01-02 06:44:22|
|     CLFTR|       Cliff Theater| 332283|          3|        4.8|7B7DF367-5D94-454...|2018-01-02 09:32:41|
|     CTPTR|     Capitol Theater| 439502|          1|        3.3|9A66D9DC-4C8E-4E8...|2018-01-02 11:21:46|
|     CSRTR|      Castro Theatre| 300665|          2|        7.8|8BD63968-EBDD-461...|2017-12-30 11:48:09|
|     CSRTR|      Castro Theatre| 460

In [8]:
excluded_cols_src = ['TransactionDt']

src_cols = [col.strip().lower() for col in src_df.columns if col not in excluded_cols_src]
src_cols

['cinemacode',
 'cinemaname',
 'movieid',
 'ticketcount',
 'ticketprice',
 'transactionid']

In [17]:
src_columns = [' ID ', ' Name', 'Amount ', 'CreatedDate']
tgt_columns = ['id', 'AMOUNT', 'name', 'UpdatedBy']
oth_columns = [' ID ', ' Name', 'AMOUNT ', 'CreatedDate', 'UpdatedBy']
excluded_columns = ['createddate', 'updatedby']

src_cols = [col.strip().lower() for col in src_columns if col.strip().lower() not in excluded_columns]
tgt_cols = [col.strip().lower() for col in tgt_columns if col.strip().lower() not in excluded_columns]
oth_cols = [col.strip().lower() for col in oth_columns if col.strip().lower() not in excluded_columns]

print(src_cols)
print(tgt_cols)
print(oth_cols)

['id', 'name', 'amount']
['id', 'amount', 'name']
['id', 'name', 'amount']


In [26]:
zipped_cols = enumerate(zip(src_cols, tgt_cols))
list(zipped_cols)

[(0, ('id', 'id')), (1, ('name', 'amount')), (2, ('amount', 'name'))]

In [27]:
mismatched = []

for idx, (src_c, tgt_c) in enumerate(zip(src_cols, tgt_cols)):
    if src_c != tgt_c:
        mismatched.append((idx, src_c, tgt_c))

In [28]:
mismatched

[(1, 'name', 'amount'), (2, 'amount', 'name')]

In [29]:
[str.lower('TransactionDt')]

['transactiondt']

In [18]:
names = ["Ricky", "Bunny", "Coco"]
ages = [10, 15, 20]

df1 = spark.createDataFrame(zip(names, ages), ["Name", "Age"])
df1.show()


Py4JJavaError: An error occurred while calling o128.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 1 times, most recent failure: Lost task 0.0 in stage 6.0 (TID 6) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadChe

In [29]:
data1 = [(1, 'a'), (1, 'a'), (2, 'b')]
data2 = [(1, 'a'), (2, 'b')]

import pandas as pd

pdf1 = pd.DataFrame(data1)
display(df1)

pdf2 = pd.DataFrame(data2)
print(df2)

df1 = spark.createDataFrame(pdf1)
df2 = spark.createDataFrame(pdf2)

,0,1
0,1,a
1,1,a
2,2,b


   0  1
0  1  a
1  2  b


In [34]:
df = df1.subtract(df2)

# df.show(2)

Py4JJavaError: An error occurred while calling o266.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 13.0 failed 1 times, most recent failure: Lost task 0.0 in stage 13.0 (TID 13) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrRead

,0,1
0,1,a
1,1,a
2,2,b


   0  1
0  1  a
1  2  b
